## Demonstrate the Aggregated Prediction Methods

In [1]:
import helper
import aggregated_predict as ap

In [2]:
sensor_test, network_test = helper.load_test_set()

local_models = ['./received_models/client_1.joblib', './received_models/client_2.joblib', './received_models/client_3.joblib']
global_model = './received_models/global_model.joblib'

model_weights = [0.15, 0.15, 0.15, 0.55]

models_predicts, metrics = ap.get_predictions_and_metrics(
    local_models=local_models,
    sensor_test=sensor_test,
    global_model=global_model,
    network_test=network_test,
)

Sensor Model 1 Prediction Results:
-----------------------------
Accuracy : 0.9954018311291963
Precision: 0.9954110674493984
Recall   : 0.9954018311291963
F1 Score : 0.9953572822033818
-----------------------------

Sensor Model 2 Prediction Results:
-----------------------------
Accuracy : 0.9953986186218343
Precision: 0.9954075363638438
Recall   : 0.9953986186218343
F1 Score : 0.9953541453841006
-----------------------------

Sensor Model 3 Prediction Results:
-----------------------------
Accuracy : 0.9952058681801146
Precision: 0.9952147379476494
Recall   : 0.9952058681801146
F1 Score : 0.9951578737262952
-----------------------------

Network Model Prediction Results:
-----------------------------
Accuracy : 0.9999132623012261
Precision: 0.999913263638906
Recall   : 0.9999132623012261
F1 Score : 0.9999132629406795
-----------------------------


In [3]:
y_test = network_test.iloc[:, -1]

In [4]:
print(f"Aggregated Prediction by Score:")
score = ap.aggregate_predict_by_score(
    models_predictions=models_predicts,
    models_metrics=metrics,
    models_weights=model_weights)
helper.get_metrics(y_test, score, printout=True)

Aggregated Prediction by Score:
-----------------------------
Accuracy : 0.9999132623012261
Precision: 0.999913263638906
Recall   : 0.9999132623012261
F1 Score : 0.9999132629406795
-----------------------------


(0.9999132623012261, 0.999913263638906, 0.9999132623012261, 0.9999132629406795)

In [5]:
print(f"Aggregated Prediction by Vote:")
vote = ap.aggregate_predict_by_vote(models_predictions=models_predicts)
helper.get_metrics(y_test, vote, printout=True)

Aggregated Prediction by Vote:
-----------------------------
Accuracy : 0.9913615677035926
Precision: 0.9914434417000387
Recall   : 0.9913615677035926
F1 Score : 0.9911812937350675
-----------------------------



(0.9913615677035926,
 0.9914434417000387,
 0.9913615677035926,
 0.9911812937350675)